In [1]:
import pickle
import pandas as pd
import csv

results = list()

datasetFile = open("fnmr/lighting/dataset/fnmr_dataset_multipie_lighting.pickle", 'rb')
dataset = pickle.load(datasetFile)

resultFile = open("fnmr/lighting/results/openface_fnmr_multipie_lighting.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/lighting/results/arcface_fnmr_multipie_lighting.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/lighting/results/facenet_fnmr_multipie_lighting.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/lighting/results/vggface_fnmr_multipie_lighting.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/lighting/results/faceapi_fnmr_multipie_lighting.pickle", 'rb')
faceapiDataset = (pickle.load(resultFile))


resultDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for result in results:
    for algorithm in result.keys():
        if result[algorithm]:
            resultDict[algorithm] = result[algorithm]

In [2]:
processDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for algorithm in resultDict.keys():
    for index in range(len(dataset)):
        referenceSubject = (dataset[index][0].split('/'))[3]
        lighting = (dataset[index][1].split('/'))[6].split('_')[4].replace(".png", "")
        
        pairNumber = 'pair_' + str(index + 1)
        if not(referenceSubject in processDict[algorithm].keys()):
            processDict[algorithm][referenceSubject] = dict()
        if not(lighting in processDict[algorithm][referenceSubject].keys()):
            processDict[algorithm][referenceSubject][lighting] = list()
        processDict[algorithm][referenceSubject][lighting].append(resultDict[algorithm][pairNumber])
    
faceAPIResult = dict()
for comparison in faceapiDataset:
    parsedComparison = comparison.split(" ")
    parsedComparison[0] = parsedComparison[0].replace("/home/ubuntu/Multipie/Multi-Pie/", "")
    parsedComparison[1] = parsedComparison[1].replace("/home/ubuntu/Multipie/Multi-Pie/", "")
    parsedComparison[2].replace("undefined", "1")
    try: 
        parsedComparison[2] = float(parsedComparison[2])
    except:
        parsedComparison[2] = float(1)

    
    referenceSubject = (parsedComparison[0].split('/'))[3]
    lighting = (parsedComparison[1].split('/'))[6].split('_')[4].replace(".png", "")
    distance = parsedComparison[2]

    comparisonDict = dict()
    comparisonDict["distance"] = distance
    if distance >= float(.6):
        comparisonDict["verified"] = False
    else:
        comparisonDict["verified"] = True

    if not(referenceSubject in faceAPIResult.keys()):
        faceAPIResult[referenceSubject] = dict()
    if not(lighting in faceAPIResult[referenceSubject].keys()):
        faceAPIResult[referenceSubject][lighting] = list()

    faceAPIResult[referenceSubject][lighting].append(comparisonDict)

processDict["FaceAPI"] = faceAPIResult


In [3]:
verifiedDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}


for algorithm in processDict.keys():
    for subject in processDict[algorithm].keys():
        if not(subject in verifiedDict[algorithm].keys()):
            verifiedDict[algorithm][subject] = dict()
        for lighting in processDict[algorithm][subject].keys():
            if not(lighting in verifiedDict[algorithm][subject].keys()):
                verifiedDict[algorithm][subject][lighting] = list()
            for comparison in processDict[algorithm][subject][lighting]:
                verified = False
                if comparison["distance"] < .6:
                    verified = True
                verifiedDict[algorithm][subject][lighting].append(verified)



In [5]:
verifiedStats = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

lightingConditions = ["00", "04", "12", "16"]

for algorithm in verifiedDict.keys():
    verifiedStats[algorithm]["tpr"] = 0
    verifiedStats[algorithm]["fnr"] = 0
    verifiedStats[algorithm]["lighting"] = dict()
    for lightingCondition in lightingConditions:
        verifiedStats[algorithm]["lighting"][lightingCondition] = dict()
        verifiedStats[algorithm]["lighting"][lightingCondition]["tpr"] = 0
        verifiedStats[algorithm]["lighting"][lightingCondition]["fnr"] = 0
    for subject in verifiedDict[algorithm].keys():
        for lighting in verifiedDict[algorithm][subject].keys():
            for comparison in verifiedDict[algorithm][subject][lighting]:
                if comparison:
                    verifiedStats[algorithm]["lighting"][lighting]["tpr"] = verifiedStats[algorithm]["lighting"][lighting]["tpr"] + 1
                    verifiedStats[algorithm]["tpr"] = verifiedStats[algorithm]["tpr"] + 1
                else:
                    verifiedStats[algorithm]["lighting"][lighting]["fnr"] = verifiedStats[algorithm]["lighting"][lighting]["fnr"] + 1
                    verifiedStats[algorithm]["fnr"] = verifiedStats[algorithm]["fnr"] + 1

Overall FPR and TNR by algorithm:

In [6]:
print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + str(verifiedStats[algorithm]["fnr"]))
    print("TP:  " + str(verifiedStats[algorithm]["tpr"]))  
    print("FNMR:  " + str(round(float(verifiedStats[algorithm]["fnr"])/float(verifiedStats[algorithm]["fnr"] + verifiedStats[algorithm]["tpr"]) * 100, 3)) + "%")  


FNMR Results:

Facenet
FN:  25761
TP:  104967
FNMR:  19.706%

VGG-Face
FN:  2974
TP:  127754
FNMR:  2.275%

OpenFace
FN:  14598
TP:  116130
FNMR:  11.167%

ArcFace
FN:  18816
TP:  111912
FNMR:  14.393%

FaceAPI
FN:  11396
TP:  119332
FNMR:  8.717%


In [8]:
for algorithm in verifiedStats.keys():
    for lighting in verifiedStats[algorithm]["lighting"].keys():
        verifiedStats[algorithm]["lighting"][lighting]["fnmr"] = round((float(verifiedStats[algorithm]["lighting"][lighting]["fnr"])/(float(verifiedStats[algorithm]["lighting"][lighting]["fnr"] + verifiedStats[algorithm]["lighting"][lighting]["tpr"]))) * 100, 3)


print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + "\t00: " + str(verifiedStats[algorithm]["lighting"]["00"]["fnr"]) + "\t04: " + str(verifiedStats[algorithm]["lighting"]["04"]["fnr"]) + "\t12: " + str(verifiedStats[algorithm]["lighting"]["12"]["fnr"]) + "\t16: " + str(verifiedStats[algorithm]["lighting"]["16"]["fnr"]))
    print("TP:  " + "\t00: " + str(verifiedStats[algorithm]["lighting"]["00"]["tpr"]) + "\t04: " + str(verifiedStats[algorithm]["lighting"]["04"]["tpr"]) + "\t12: " + str(verifiedStats[algorithm]["lighting"]["12"]["tpr"]) + "\t16: " + str(verifiedStats[algorithm]["lighting"]["16"]["tpr"]))
    print("FNMR:  " + "\t00: " + str(verifiedStats[algorithm]["lighting"]["00"]["fnmr"]) + "%\t04: " + str(verifiedStats[algorithm]["lighting"]["04"]["fnmr"]) + "%\t12: " + str(verifiedStats[algorithm]["lighting"]["12"]["fnmr"]) + "%\t16: " + str(verifiedStats[algorithm]["lighting"]["16"]["fnmr"]) + "%")

FNMR Results:

Facenet
FN:  	00: 11174	04: 5028	12: 5430	16: 4129
TP:  	00: 21508	04: 27654	12: 27252	16: 28553
FNMR:  	00: 34.19%	04: 15.385%	12: 16.615%	16: 12.634%

VGG-Face
FN:  	00: 361	04: 870	12: 775	16: 968
TP:  	00: 32321	04: 31812	12: 31907	16: 31714
FNMR:  	00: 1.105%	04: 2.662%	12: 2.371%	16: 2.962%

OpenFace
FN:  	00: 4281	04: 3769	12: 3790	16: 2758
TP:  	00: 28401	04: 28913	12: 28892	16: 29924
FNMR:  	00: 13.099%	04: 11.532%	12: 11.597%	16: 8.439%

ArcFace
FN:  	00: 4885	04: 4987	12: 4813	16: 4131
TP:  	00: 27797	04: 27695	12: 27869	16: 28551
FNMR:  	00: 14.947%	04: 15.259%	12: 14.727%	16: 12.64%

FaceAPI
FN:  	00: 5435	04: 2356	12: 2673	16: 932
TP:  	00: 27247	04: 30326	12: 30009	16: 31750
FNMR:  	00: 16.63%	04: 7.209%	12: 8.179%	16: 2.852%
